#### Client의 행동을 따라가보자
- Client가 어떻게 행동하는지 데이터 살펴보기
- Client Behavior 데이터 만들기 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#구매데이터의 client 수 != 고객데이터의 client 수 
#구매데이터의 client 수 != 검색데이터의 client 수
print('Product shape: ',Product.shape)
print('Custom shape: ',Custom.shape)
print('Search1 shape: ',Search1.shape)
print('Session shape: ',Session.shape)
print()
print('구매에 참여한 client 수: ',len(Product['CLNT_ID'].value_counts()))
print('고객데이터에 있는 unique한 client 수: ',len(Custom['CLNT_ID'].value_counts()))
print('세션에 참여한 unique한 client 수: ',len(Session['CLNT_ID'].value_counts()))
print('검색에 참여한 client 수: ', len(Search1['CLNT_ID'].value_counts()))

Product shape:  (5024906, 8)
Custom shape:  (671679, 3)
Search1 shape:  (2884943, 4)
Session shape:  (2712907, 9)

구매에 참여한 client 수:  922737
고객데이터에 있는 unique한 client 수:  671679
세션에 참여한 unique한 client 수:  922737
검색에 참여한 client 수:  511477


- 세션에 참여한 client 수와 구매에 참여한 client 수는 동일하다 
- 상품을 구매했을지라도 검색을 하지 않은 client들이 있다 ) 922737 - 511477 만큼
- 고객데이터에 모든 client들의 정보가 있는 것은 아니다 

In [4]:
temp1 = pd.merge(Search1, Product, how = 'right' , on = 'CLNT_ID')

In [5]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [6]:
temp2 = pd.merge(Custom, Product, how = 'right' , on = 'CLNT_ID')

In [7]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [8]:
#null값 체크 --> 이건 unique한 건 아니지만 그래도 null값이 있다는 걸 의미함 
print(temp2['CLNT_AGE'].isnull().sum().sum())

1036319


- Client ID, 날짜(세션일자), 세션 ID, HIT번호, 검색키워드, 검색건수, 상품명, 구매건수 

- CLNT_ID(Product), SESS_DT(Session), SESS_ID(세션ID), HITS_SEQ(Product)
- KWD_NM(Search1), SEARCH_CNT(Search1), PD_NM(Master), PD_BUY_CT(Product)

#### Search1의 client < custom의 client < Product의 client = Session의 Client
#### Search1의 client 중에 custom 정보 없는 것 있다. 
#### 검색을 하고, custom 정보가 있는 client만을 기준으로 table 작성 (375864개의 client id)

In [102]:
#Search1의 client 중에 custom 정보 없는 것 있다. 
temp = pd.merge(Custom, Search1, how = 'inner', on = 'CLNT_ID')
print(len(temp['CLNT_ID'].value_counts()))

375864


- 데이터 전처리 : outlier 제거
- HIT_SEQ가 1인 것 제거하기 

In [27]:
#HIT_SEQ가 1인 Column 제거하기 (outlier로 간주하라고 하였음)

idx = (Product['HITS_SEQ'] != 1)
Product = Product[idx]
print(len(Product['CLNT_ID'].value_counts())) 

922618


In [28]:
Product[Product['CLNT_ID'] == 4758889]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
533815,4758889,10607258,24,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
597057,4758889,10607258,195,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
655687,4758889,10607258,298,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
661037,4758889,10730334,76,692624,색상:블랙 / 1개,LG전자,"45,340",1
4954520,4758889,10607258,298,776358,사이즈:28 / 1개,게스(진),"39,000",1
5022807,4758889,10607258,195,776358,사이즈:28 / 1개,게스(진),"39,000",1


In [39]:
#Product에서 outlier를 제거해 줬으므로 Product의 client에 맞춘다
data1 = pd.merge(Product, Session)
print(len(data1['CLNT_ID'].value_counts()))
print('아주 조금 줄었음')

922618
아주 조금 줄었음


In [30]:
#상품정보를 추가해준다
data2 = pd.merge(data1, Master, how = 'left', on = 'PD_C')
print(len(data1['CLNT_ID'].value_counts()))

922618


In [17]:
#세션과 구매에 참여한 client를 기준으로 data2과 Search1을 left outer join 해 준다 
#세션과 구매에 참여한 client일지라도 검색을 하지 않은 client가 존재한다
#data2 = data2[['CLNT_ID','SESS_ID','HITS_SEQ','SESS_DT','TOT_PAG_VIEW_CT','PD_NM','CLAC1_NM','PD_BUY_CT','ZON_NM']]

In [40]:
idx = data2['CLNT_ID'] == 4758889
data2[idx]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
966746,4758889,10607258,24,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966747,4758889,10607258,195,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966748,4758889,10607258,298,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966749,4758889,10607258,298,776358,사이즈:28 / 1개,게스(진),"39,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,여성 원워시 슈퍼스키니(YG1D9043) - 26,남성의류,남성의류하의,남성캐주얼바지
966750,4758889,10607258,195,776358,사이즈:28 / 1개,게스(진),"39,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,여성 원워시 슈퍼스키니(YG1D9043) - 26,남성의류,남성의류하의,남성캐주얼바지
1176448,4758889,10730334,76,692624,색상:블랙 / 1개,LG전자,"45,340",1,75,20180404,41.0,"1,134",mobile,Gyeonggi-do,Paju-si,[공식인증점]HBS-510 톤플러스(TONE PLUS) 블루투스 이어폰 /초경량디자...,모바일,모바일액세서리,이어폰/헤드폰


In [35]:
#Search1의 client기준으로 병합
#client -> 검색 -> 구매
Client_Behavior = pd.merge(data2, Search1, on=['CLNT_ID', 'SESS_ID']) 

In [37]:
print(len(Client_Behavior['CLNT_ID'].value_counts()))

511447


In [103]:
#Custom의 client와 겹치는 부분만 select
Client_Behavior = pd.merge(Client_Behavior, Custom, on = 'CLNT_ID')

In [105]:
print(len(Client_Behavior['CLNT_ID'].value_counts()))

375839


In [107]:
Client_Behavior.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,...,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT,CLNT_GENDER,CLNT_AGE
0,4147643,4050743,66,149995,앵글로매니아,"39,000",1,18,20180724,73.0,...,Seoul,Seoul,[ANGLOMANIA][5종택1] 명품 선글라스 5 - 옵션03.AN868_03,패션잡화,안경/선글라스,여성선글라스,선글라스,1,F,30
1,4147643,4050743,66,149995,앵글로매니아,"39,000",1,18,20180724,73.0,...,Seoul,Seoul,[ANGLOMANIA][5종택1] 명품 선글라스 5 - 옵션03.AN868_03,패션잡화,안경/선글라스,여성선글라스,앵글로매니아,1,F,30
2,4159738,3278944,42,184776,레노마수영복,"39,000",1,488,20180806,145.0,...,Seoul,Seoul,[롯데단독] 여성 비치 심플 워터레깅스 팬츠(+네임텍)_WS18501 - BK / 85,시즌스포츠,수영/물놀이,여성비치웨어,견미리팩트,1,F,40
3,4159738,3278944,131,838568,필립스(PHILIPS),"49,000",1,488,20180806,145.0,...,Seoul,Seoul,[데일리 미니 커피메이커 HD7434/20],생활/주방가전,주방가전,커피메이커/포트,견미리팩트,1,F,40
4,4159738,3278944,131,184776,레노마수영복,"39,000",1,488,20180806,145.0,...,Seoul,Seoul,[롯데단독] 여성 비치 심플 워터레깅스 팬츠(+네임텍)_WS18501 - BK / 85,시즌스포츠,수영/물놀이,여성비치웨어,견미리팩트,1,F,40


In [108]:
#필요 없는 column 제거
Client_Behavior = Client_Behavior.drop(Client_Behavior[['PD_ADD_NM','DVC_CTG_NM']], axis=1) 

KeyError: "['PD_ADD_NM' 'DVC_CTG_NM'] not in index"

In [42]:
#Client_Behavior 데이터 내보내기
Client_Behavior.to_csv("C:/Users/lsh93/Desktop/data/Client_Behavior.csv", encoding="UTF-8")

### 상품 구매량 count를 위한 테이블 만들기

- 상품 구매 count를 하기 위해서, Product에서 HIT_SEQ = 1을 제거하고(앞에서 이미 제거됨), Search1과 custom에 둘 다 존재하는 Client 기준으로 정렬
- 한 client의 hit번호가 가장 큰 것에 대해서만 따로 빼서 매트릭스를 만들고, 그 후 상품분류에 대해 value_counts()해 주면 될듯

In [110]:
#Search1의 client를 unique하게 만들어주기 
Client = pd.merge(Search1[['CLNT_ID']], Custom[['CLNT_ID']])
Unique_Client = Client.drop_duplicates(subset=None, keep='last', inplace=False)

In [111]:
Unique_Client['CLNT_ID'].value_counts()

5461714    1
557637     1
5280327    1
5328413    1
2685514    1
6353483    1
4246092    1
4768333    1
55886      1
2150991    1
1583696    1
1581649    1
5767765    1
5773910    1
2365283    1
1079898    1
553563     1
5784157    1
6314590    1
6400608    1
2380969    1
2136646    1
3191363    1
4842089    1
2772219    1
4954662    1
2855463    1
2353704    1
1303081    1
5857480    1
          ..
4580630    1
2987288    1
2465049    1
4762753    1
6130971    1
1081954    1
4589347    1
5615932    1
4015362    1
4740254    1
1896362    1
1211621    1
6448358    1
1532795    1
5179014    1
3592504    1
4859118    1
3331312    1
5954801    1
6487285    1
5141990    1
4908278    1
6505990    1
2266360    1
5938425    1
6448371    1
4371708    1
3319038    1
6466815    1
6291456    1
Name: CLNT_ID, Length: 375864, dtype: int64

In [112]:
#Search1과 custom에 공통으로 있는 client id에 해당하는 구매내역만 dataframe으로 만들기 
Product_Client = pd.merge(Product, Unique_Client, how = 'inner', on = 'CLNT_ID')
print(len(Product_Client['CLNT_ID'].value_counts()))   

375864


In [115]:
Product_Client = Product_Client.drop_duplicates(subset=None, keep='last', inplace=False)
print(len(Product_Client['CLNT_ID'].value_counts()))   

375864


In [116]:
Product_Client

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
1,4140076,10342512,23,422358,선택:02.204143-02S|사이즈:M5W7(240) / 1개,크록스,"39,900",1
2,4140076,6526651,22,196272,색상:선택3)248(네이비)|사이즈:L(105) / 1개,프레드페리,"47,600",1
3,4140076,8326529,199,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
4,4140076,8891563,19,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
5,4140076,6526651,193,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 1개,프레드페리,"47,600",1
6,4140076,6591360,149,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 2개,프레드페리,"47,600",2
7,4140076,6526651,22,365025,색상:WH|사이즈:165 / 1개,닥스 키즈,"48,300",1
8,4140076,4756252,147,794908,1개,네이쳐리빙,"7,200",1
9,4140076,10342512,23,403243,사이즈:M9 / 1개,크록스,"57,000",1
